In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from fe import *

import numpy as np
import pandas as pd
import collections as col

In [3]:
def smape_cal(y_true, y_pred):
    smape = (np.abs(y_true-y_pred)*200)/(np.abs(y_true)+np.abs(y_pred))
    return smape

def mase_cal(y_true, y_pred, freq, len_series, timeseries):
    forecastsNaiveSD = [np.nan]*freq
    for j in range(freq, len_series):
        forecastsNaiveSD.append(timeseries[j - freq])
        
    masep = np.nanmean(np.abs(timeseries - forecastsNaiveSD))
  
    mase = (np.abs(y_true-y_pred)) / masep
    return(mase)

In [5]:
df = pd.read_csv('data/ETHBTC_1517303160000_1547498100000.csv')

In [6]:
df.head()

OPEN_TIME      OPEN      HIGH       LOW     CLOSE       VOLUME  \
0  1517303100000  10629400  10673700  10629300  10640100  17785000000   
1  1517303160000  10640100  10643300  10619800  10630100  15433400000   
2  1517303220000  10621100  10640000  10621000  10625000   9354600000   
3  1517303280000  10626200  10634500  10622600  10630800   6746400000   
4  1517303340000  10630800  10639000  10622600  10632600   7650900000   

      CLOSE_TIME   QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1517303159999  1892484646         412    10429400000      1110434538      0  
1  1517303219999  1640768918         276     4288000000       456085864      0  
2  1517303279999   994314138         255     4298600000       457129449      0  
3  1517303339999   716919384         184     3005400000       319476265      0  
4  1517303399999   813139316         182     3471000000       369112330      0

In [7]:
data = df

ohlc_feat        = ohlc_feats(data)
ohlc_cross_feat  = ohlc_cross_feats(data)
vol_feat         = vol_feats(data)
ohlc_vol_feat    = ohlc_vol_feats(data)
num_trades_feat  = num_trades_feats(data)

all_feat = ohlc_feat.merge(ohlc_cross_feat, on='OPEN_TIME')
all_feat = all_feat.merge(vol_feat, on='OPEN_TIME')
all_feat = all_feat.merge(ohlc_vol_feat, on='OPEN_TIME')
all_feat = all_feat.merge(num_trades_feat, on='OPEN_TIME')

fe.py:59: RuntimeWarning: invalid value encountered in divide
  ohlc_feats['O_PERCENTILE']            = (o - l)/(h - l)
fe.py:60: RuntimeWarning: invalid value encountered in divide
  ohlc_feats['C_PERCENTILE']            = (c - l)/(h - l)
fe.py:143: RuntimeWarning: invalid value encountered in divide
  vol_feats['BV_QV_FRAC'    ] = bv / qv
fe.py:144: RuntimeWarning: invalid value encountered in divide
  vol_feats['QV_BV_FRAC'    ] = qv / bv
fe.py:145: RuntimeWarning: invalid value encountered in divide
  vol_feats['BV_QV_FRAC_INV'] = 1 - bv / qv
fe.py:146: RuntimeWarning: invalid value encountered in divide
  vol_feats['QV_BV_FRAC_INV'] = 1 - qv / bv
fe.py:148: RuntimeWarning: invalid value encountered in divide
  vol_feats['TBV_TQV_FRAC'    ] = tbv / tqv
fe.py:149: RuntimeWarning: invalid value encountered in divide
  vol_feats['TQV_TBV_FRAC'    ] = tqv / tbv
fe.py:150: RuntimeWarning: invalid value encountered in divide
  vol_feats['TBV_TQV_FRAC_INV'] = 1 - tbv / tqv
fe.py:151: Runt

In [8]:
all_feat.shape
all_feat.head()

(499999, 145)

OPEN_TIME     OHLC     OLHC O_EQUAL_C O_EQUAL_L O_EQUAL_H C_EQUAL_H  \
0  1517303160000  37000.0  57000.0     False     False     False     False   
1  1517303220000  41900.0  34100.0     False     False     False     False   
2  1517303280000  28400.0  19200.0     False     False     False     False   
3  1517303340000  34600.0  31000.0     False     False     False     False   
4  1517303400000  15000.0  35400.0     False     False      True     False   

  C_EQUAL_L L_EQUAL_H O_GREATER_C        ...          NUM_TRADES_PER_TBV  \
0     False     False        True        ...                6.436567e-08   
1     False     False       False        ...                5.932164e-08   
2     False     False       False        ...                6.122313e-08   
3     False     False       False        ...                5.243446e-08   
4     False     False        True        ...                6.591662e-08   

   NUM_TRADES_PER_TQV  BV_PER_NUM_TRADES  QV_PER_NUM_TRADES  \
0        6.051492e-07       5.591812e+07       5.944815e+06   
1        5.578289e-07       3.668471e+07       3.899271e+06   
2        5.759426e-07       3.666522e+07       3.896301e+06   
3        4.930748e-07       4.203791e+07       4.467798e+06   
4        6.203546e-07       3.456702e+07       3.672382e+06   

   TBV_PER_NUM_TRADES  TQV_PER_NUM_TRADES  NUM_TRADES_HL_SPREAD  \
0        1.553623e+07        1.652485e+06          1.060482e+07   
1        1.685725e+07        1.792665e+06          1.059835e+07   
2        1.633370e+07        1.736284e+06          1.057677e+07   
3        1.907143e+07        2.028090e+06          1.058063e+07   
4        1.517068e+07        1.611981e+06          1.057700e+07   

   NUM_TRADES_HL_SPREAD_INV  NUM_TRADES_HL_AVG  NUM_TRADES_OC_AVG  
0             -1.061980e+07       38520.108696       38532.971014  
1             -1.062100e+07       41688.235294       41659.019608  
2             -1.062260e+07       57763.858696       57763.586957  
3             -1.062260e+07       58410.989011       58415.934066  
4             -1.062000e+07       55635.078534       55641.361257  

[5 rows x 145 columns]

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import Imputer

Y = all_feat.loc[:, 'O_GREATER_C'].values[1:]
X = all_feat.values[:-1]

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

In [11]:
window_size = 5

X_ = np.zeros((X.shape[0] - window_size + 1, X.shape[1]*window_size))
Y_ = np.zeros((X.shape[0] - window_size + 1, ))

for i in range(window_size):
    X_[:, i*X.shape[1]:(i+1)*X.shape[1]] = X[i:X.shape[0]-window_size+i+1]

Y_ = Y[window_size-1:]

model_ = RandomForestClassifier(n_estimators=30, criterion='gini', n_jobs=-1, max_features=50, random_state=0, verbose=1)
model_ = model_.fit(X_[:300000], Y_[:300000])

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   19.1s finished


In [10]:
# Modelling
cv = 5
scores = [0]*cv

for cv_fold in range(cv):
    model = RandomForestClassifier(n_estimators=3, criterion='gini', n_jobs=-1, max_features=None, random_state=0, verbose=1)
    model = model.fit(X[:len(X) - (cv-cv_fold)*50000], Y[:len(X) - (cv-cv_fold)*50000])
    Y_pred = model.predict(X[len(X) - (cv-cv_fold)*50000:len(X) - (cv-cv_fold)*50000 + 50000])
    scores[cv_fold] = (Y[len(X) - (cv-cv_fold)*50000:len(X) - (cv-cv_fold)*50000 + 50000:] == Y_pred).sum()/50000.0
    print cv_fold, scores[cv_fold]

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.6min finished
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.2s finished


0 0.5078


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  8.4min finished
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.2s finished


1 0.50358


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 10.7min finished
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.3s finished


...

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 12.9min finished
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.3s finished


3 0.50532


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 15.1min finished


4 0.50186


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.4s finished


In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'output/MODEL1.model') 

In [ ]:
for i in range(len(model.feature_importances_)):
    print(all_feats.columns[i], model.feature_importances_[i])

In [ ]:
model.feature_importances_

In [ ]:
Y_pred = model.predict(X[300000:])
(Y[300000:] == Y_pred).sum()

In [12]:
Y_pred = model_.predict(X_[300000:])
(Y_[300000:] == Y_pred).sum()

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


100377